# Google Search
> Fill in a module description here

In [1]:
# | default_exp google.GSearch

In [2]:
# | exporti
from typing import List
from dataclasses import dataclass, field
import gdoc_sync.client as gd
import httpx

from nbdev.showdoc import patch_to

In [3]:
# | hide
from nbdev.showdoc import show_doc
from dotenv import load_dotenv
import os

assert load_dotenv("../.env", override=True)

In [4]:
# | export
@dataclass
class GoogleSearchAuth(gd.Auth):
    search_key: str
    search_engine_id: str
    base_url: str = None

    def __post_init__(self):
        self.base_url = f"https://www.googleapis.com/customsearch/v1?cx={self.search_engine_id}&key={self.search_key}"

    def generate_auth_header(self) -> dict:
        return self.auth_header

In [5]:
# | export
@dataclass
class GoogleSearch:
    auth: GoogleSearchAuth = field(repr=False)
    query: str
    is_from_cache: bool
    cnt_results: int
    content: List[dict]
    raw_content: dict = field(repr=False)

    @classmethod
    async def search(
        cls,
        auth: GoogleSearchAuth,
        query: str,
        json_cache_path: str = None,
        # return_raw: bool = False,
        is_ignore_cache: bool = False,
        debug_api: bool = False,
        client: httpx.AsyncClient = None,
    ):
        res = await gd.get_data(
            url=auth.base_url,
            method="GET",
            json_cache_path=json_cache_path,
            debug_api=debug_api,
            params={"q": query},
            is_ignore_cache=is_ignore_cache,
            client=client,
        )

        return cls(
            auth=auth,
            query=query,
            is_from_cache=res.is_from_cache,
            cnt_results=len(res.response["items"]),
            raw_content=res.response,
            content=res.response["items"],
        )

In [6]:
gs_auth = GoogleSearchAuth(
    search_key=os.environ["GDOC_SEARCH_KEY"],
    search_engine_id=os.environ["GDOC_SEARCH_ENGINE"],
)


gs_search = await GoogleSearch.search(
    auth=gs_auth, query="datacrew", json_cache_path="../TEST/search/datacrew.json"
)

gs_search

GoogleSearch(query='datacrew', is_from_cache=True, cnt_results=10, content=[{'kind': 'customsearch#result', 'title': 'Data Crew | Inspired by Hip Hop & American Pop culture– datacrew', 'htmlTitle': '<b>Data Crew</b> | Inspired by Hip Hop &amp; American Pop culture– <b>datacrew</b>', 'link': 'https://datacrewla.com/', 'displayLink': 'datacrewla.com', 'snippet': 'Featuring Pins and Apparel designs from Artists around the world.', 'htmlSnippet': 'Featuring Pins and Apparel designs from Artists around the world.', 'cacheId': '3GVPqp4ExIAJ', 'formattedUrl': 'https://datacrewla.com/', 'htmlFormattedUrl': 'https://<b>datacrew</b>la.com/', 'pagemap': {'offer': [{'pricecurrency': 'USD', 'price': '12.0', 'availability': 'http://schema.org/InStock'}], 'cse_thumbnail': [{'src': 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSOhKgI6xYFTTDSreAXiyxIqd58ygXeOUfgEpC9sqTADn-cqX7pkbZAMTI', 'width': '347', 'height': '145'}], 'product': [{'image': '//datacrewla.com/cdn/shop/products/Data_DefJam_Ble

In [7]:
# | hide
import nbdev

nbdev.nbdev_export()

JSONDecodeError: Expecting value: line 1 column 1 (char 0)